In [8]:
from datasets import load_dataset, Dataset
import torch
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    get_linear_schedule_with_warmup,
)
from peft import (
    PeftModel,
    LoraConfig,
    TaskType,
    get_peft_model,
    prepare_model_for_kbit_training,
)
from tqdm import tqdm
import torch.nn.functional as F
from torch.utils.data import DataLoader
from typing import Literal
import random
import numpy as np

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [9]:
def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)


set_seed(42)

In [10]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
)
peft_config = LoraConfig(
    r=8,
    target_modules=[
        "q_proj",
        "v_proj",
        "k_proj",
        "o_proj",
        "gate_proj",
        "down_proj",
        "up_proj",
    ],
    task_type=TaskType.CAUSAL_LM,
    lora_alpha=16,
    lora_dropout=0.05,
)

In [11]:
model_path = "../model_save/base_model/qwen-1.5-1.8b/"
tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)
model_policy = AutoModelForCausalLM.from_pretrained(
    model_path,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True,
)
model_policy = prepare_model_for_kbit_training(model_policy)
model_policy = get_peft_model(model_policy, peft_config)
model_policy.print_trainable_parameters()
model_ref = AutoModelForCausalLM.from_pretrained(
    model_path,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True,
)

trainable params: 7,495,680 || all params: 1,844,324,352 || trainable%: 0.4064


In [13]:
dataset = load_dataset("json", data_files="./dataset/ultrafeedback/flan.jsonl")

Generating train split: 0 examples [00:00, ? examples/s]

In [14]:
dataset

DatasetDict({
    train: Dataset({
        features: ['source', 'instruction', 'models', 'completions', 'correct_answers', 'incorrect_answers'],
        num_rows: 20939
    })
})

In [15]:
def manual_pad_batch_tensors(
    tensors_list,
    padding_value=0,
    padding_side: Literal["left", "right"] = "left",
):
    """手动实现批次张量的填充"""
    # 获取最大序列长度
    max_seq_len = max(t.shape[1] for t in tensors_list)

    # 创建填充后的张量列表
    padded_tensors = []
    for tensor in tensors_list:
        b, s = tensor.shape
        # 创建填充张量
        padding = torch.full(
            (b, max_seq_len - s),
            padding_value,
            dtype=tensor.dtype,
            device=tensor.device,
        )
        # 拼接原始张量和填充部分
        if padding_side == "left":
            padded = torch.cat([padding, tensor], dim=1)
        elif padding_side == "right":
            # 如果是右侧填充，则将填充部分放在后面
            padded = torch.cat([tensor, padding], dim=1)
        padded_tensors.append(padded)

    return torch.cat(padded_tensors, dim=0)


def sample_trans(sample):
    question = sample["instruction"]
    completions = sample["completions"]
    scores_list = [[0, 0], [0, 1], [0, 2], [0, 3]]
    responses = []
    for i, annotation in enumerate(completions):
        responses.append(annotation["response"])
        scores_list[i][0] = float(annotation["overall_score"])
    scores_list = sorted(scores_list, key=lambda x: x[0], reverse=True)
    results = []
    for i in range(1, 4):
        results.append(
            {
                "question": question,
                "chosen": responses[scores_list[0][1]],
                "rejected": responses[scores_list[i][1]],
            }
        )
    return results


def process_function(sample, tokenizer):
    query = sample["question"]
    chosen = sample["chosen"]
    rejected = sample["rejected"]
    chosen_inputs = tokenizer(
        "\n".join([query, chosen]),
        return_tensors="pt",
        padding=True,
        truncation=True,
        max_length=512,
    )
    query_len = len(tokenizer(query)["input_ids"])
    chosen_answer_len = chosen_inputs["input_ids"][0].shape[0] - query_len
    chosen_answer_len = max(chosen_answer_len, 0)
    rejected_inputs = tokenizer(
        "\n".join([query, rejected]),
        return_tensors="pt",
        padding=True,
        truncation=True,
        max_length=512,
    )
    rejected_answer_len = rejected_inputs["input_ids"][0].shape[0] - query_len
    rejected_answer_len = max(rejected_answer_len, 0)
    return {
        "chosen": chosen_inputs,
        "rejected": rejected_inputs,
        "chosen_answer_len": chosen_answer_len,
        "rejected_answer_len": rejected_answer_len,
    }


def collate_fn(batch, tokenizer):
    chosen_input_ids = [torch.LongTensor(s["chosen"]["input_ids"][0]) for s in batch]
    chosen_attention_mask = [
        torch.tensor(s["chosen"]["attention_mask"][0]) for s in batch
    ]
    rejected_input_ids = [
        torch.LongTensor(s["rejected"]["input_ids"][0]) for s in batch
    ]
    rejected_attention_mask = [
        torch.tensor(s["rejected"]["attention_mask"][0]) for s in batch
    ]
    chosen_inputs = tokenizer.pad(
        {
            "input_ids": chosen_input_ids,
            "attention_mask": chosen_attention_mask,
        },
        return_tensors="pt",
        padding=True,
        padding_side="left",
    )
    rejected_inputs = tokenizer.pad(
        {
            "input_ids": rejected_input_ids,
            "attention_mask": rejected_attention_mask,
        },
        return_tensors="pt",
        padding=True,
        padding_side="left",
    )
    answer_mask_chosen = torch.zeros_like(chosen_inputs["input_ids"])
    answer_mask_rejected = torch.zeros_like(rejected_inputs["input_ids"])
    for i, s in enumerate(batch):
        answer_mask_chosen[
            i, answer_mask_chosen.shape[-1] - 1 - s["chosen_answer_len"] :
        ] = 1
        answer_mask_rejected[
            i, answer_mask_chosen.shape[1] - 1 - s["rejected_answer_len"] :
        ] = 1
    return {
        "chosen_inputs": chosen_inputs,
        "rejected_inputs": rejected_inputs,
        "answer_mask_chosen": answer_mask_chosen,
        "answer_mask_rejected": answer_mask_rejected,
    }

In [16]:
text = "The capital of France is"
tokenizer(text, padding=True, return_tensors="pt")

{'input_ids': tensor([[ 785, 6722,  315, 9625,  374]]), 'attention_mask': tensor([[1, 1, 1, 1, 1]])}

In [17]:
train_dataset = dataset["train"].filter(lambda x: len(x["instruction"]) < 200)

Filter:   0%|          | 0/20939 [00:00<?, ? examples/s]

In [18]:
processed_dataset = []
for sample in tqdm(train_dataset.select(range(1000))):
    processed_dataset.extend(sample_trans(sample))

100%|██████████| 1000/1000 [00:00<00:00, 1579.17it/s]


In [19]:
new_dataset = Dataset.from_list(processed_dataset)

In [20]:
new_dataset = new_dataset.map(
    lambda sample: process_function(sample, tokenizer),
    batched=False,
    remove_columns=["question"],
)

Map:   0%|          | 0/3000 [00:00<?, ? examples/s]

In [21]:
dataloader = DataLoader(
    new_dataset, batch_size=8, collate_fn=lambda x: collate_fn(x, tokenizer)
)

In [22]:
def calculate_seq_log_prob(model, input_ids, attention_mask, answer_mask):
    logits = model(input_ids=input_ids, attention_mask=attention_mask).logits
    log_probs = F.log_softmax(logits, dim=-1)
    log_probs = log_probs[:, :-1, :]
    target_ids = input_ids[:, 1:]
    target_log_probs = log_probs.gather(-1, target_ids.unsqueeze(-1)).squeeze(-1)
    answer_mask = answer_mask[:, 1:]
    target_log_probs = target_log_probs * answer_mask
    return target_log_probs.sum(dim=-1)

In [23]:
def z_zero_estimate(policy_chosen, policy_rejected, ref_chosen, ref_rejected):
    chosen_log_prob_diff = policy_chosen - ref_chosen  # [b]
    ref_log_prob_diff = policy_rejected - ref_rejected  # [b]
    log_prob_diff_all = torch.cat([chosen_log_prob_diff, ref_log_prob_diff], dim=0)
    z_zero = log_prob_diff_all.mean()
    return z_zero

In [24]:
def kto_loss(
    policy_chosen,
    policy_rejected,
    ref_chosen,
    ref_rejected,
    lambda_d,
    lambda_u,
    beta=0.1,
):
    z_zero = z_zero_estimate(policy_chosen, policy_rejected, ref_chosen, ref_rejected)
    # 计算chosen 部分的损失
    r_theta = policy_chosen - ref_chosen
    loss_d = lambda_d * torch.sigmoid(beta * (r_theta - z_zero))
    # 计算rejected 部分的损失
    r_theta = policy_rejected - ref_rejected
    loss_u = lambda_u * torch.sigmoid(beta * (z_zero - r_theta))
    loss = (loss_d + loss_u).sum() / (len(loss_d) + len(loss_u))
    return loss

In [25]:
def train_llm_kto(
    policy_model,
    ref_model,
    train_data,
    optimizer,
    epochs=3,
    lambda_d=1.0,
    lambda_u=1.0,
    beta=0.1,
):
    data_loader = DataLoader(train_data, batch_size=8, shuffle=True)
    lr_scheduler = get_linear_schedule_with_warmup(
        optimizer,
        num_warmup_steps=0,
        num_training_steps=len(dataloader) * epochs,
    )
    policy_model.train()
    ref_model.eval()
    for epoch in range(epochs):
        total_loss = 0.0
        for batch in tqdm(dataloader):
            optimizer.zero_grad()
            chosen_ids = batch["chosen_inputs"]["input_ids"].to(device)
            rejected_ids = batch["rejected_inputs"]["input_ids"].to(device)
            attention_mask_chosen = batch["chosen_inputs"]["attention_mask"].to(device)
            attention_mask_rejected = batch["rejected_inputs"]["attention_mask"].to(
                device
            )
            answer_mask_chosen = batch["answer_mask_chosen"].to(device)
            answer_mask_rejected = batch["answer_mask_rejected"].to(device)
            policy_chosen_log_probs = calculate_seq_log_prob(
                policy_model, chosen_ids, attention_mask_chosen, answer_mask_chosen
            )
            policy_rejected_log_probs = calculate_seq_log_prob(
                policy_model,
                rejected_ids,
                attention_mask_rejected,
                answer_mask_rejected,
            )
            with torch.no_grad():
                ref_chosen_log_probs = calculate_seq_log_prob(
                    ref_model, chosen_ids, attention_mask_chosen, answer_mask_chosen
                )
                ref_rejected_log_probs = calculate_seq_log_prob(
                    ref_model,
                    rejected_ids,
                    attention_mask_rejected,
                    answer_mask_rejected,
                )
            loss = kto_loss(
                policy_chosen=policy_chosen_log_probs,
                policy_rejected=policy_rejected_log_probs,
                ref_chosen=ref_chosen_log_probs,
                ref_rejected=ref_rejected_log_probs,
                lambda_d=lambda_d,
                lambda_u=lambda_u,
                beta=beta,
            )
            loss.backward()
            optimizer.step()
            del policy_chosen_log_probs, policy_rejected_log_probs
            del ref_chosen_log_probs, ref_rejected_log_probs
            torch.cuda.empty_cache()
            lr_scheduler.step()
            total_loss += loss.item()
        print(f"Epoch {epoch+1}, Loss: {total_loss/len(data_loader):.4f}")

In [26]:
optimizer = torch.optim.AdamW(model_policy.parameters(), lr=1e-5)
epochs = 3
train_llm_kto(model_policy, model_ref, new_dataset, optimizer, epochs)

  0%|          | 0/375 [00:00<?, ?it/s]You're using a Qwen2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
/root/miniconda3/lib/python3.12/site-packages/torch/_dynamo/eval_frame.py:838: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
  3%|▎         | 12/375 [00:25<12:47,  2.12s/it]


KeyboardInterrupt: 